# Explore here

In [1]:
import pandas as pd
import chardet
with open("/workspaces/app_fk/data/tasa_vih_edad_2014-2023_4.csv", 'rb') as f:
       result = chardet.detect(f.read())
       print(result)


{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [2]:
total_data = pd.read_csv("/workspaces/app_fk/data/tasa_vih_edad_2014-2023_4.csv", encoding='ISO-8859-1', sep = ";")
total_data.head()

,id_sexo,sexo,id_grupo_etario,grupo_etario,anio,tasa_vih
0,3,Ambos sexos,10,total_grupo,2014,"15,6"
1,3,Ambos sexos,10,total_grupo,2015,"15,6"
2,3,Ambos sexos,10,total_grupo,2016,"14,6"
3,3,Ambos sexos,10,total_grupo,2017,"11,8"
4,3,Ambos sexos,10,total_grupo,2018,"13,6"


In [3]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_sexo          300 non-null    int64 
 1   sexo             300 non-null    object
 2   id_grupo_etario  300 non-null    int64 
 3   grupo_etario     300 non-null    object
 4   anio             300 non-null    int64 
 5   tasa_vih         300 non-null    object
dtypes: int64(3), object(3)
memory usage: 14.2+ KB


In [4]:
total_data.shape

(300, 6)

In [5]:
if total_data.duplicated().sum():
    total_data = total_data.drop_duplicates()
print(total_data.shape)
total_data.head()

(300, 6)


,id_sexo,sexo,id_grupo_etario,grupo_etario,anio,tasa_vih
0,3,Ambos sexos,10,total_grupo,2014,"15,6"
1,3,Ambos sexos,10,total_grupo,2015,"15,6"
2,3,Ambos sexos,10,total_grupo,2016,"14,6"
3,3,Ambos sexos,10,total_grupo,2017,"11,8"
4,3,Ambos sexos,10,total_grupo,2018,"13,6"


In [6]:
total_data.drop(["id_sexo", "id_grupo_etario"], axis = 1, inplace = True)
total_data.head()

,sexo,grupo_etario,anio,tasa_vih
0,Ambos sexos,total_grupo,2014,"15,6"
1,Ambos sexos,total_grupo,2015,"15,6"
2,Ambos sexos,total_grupo,2016,"14,6"
3,Ambos sexos,total_grupo,2017,"11,8"
4,Ambos sexos,total_grupo,2018,"13,6"


In [7]:
print(total_data['sexo'].unique())
print(total_data['grupo_etario'].unique())

['Ambos sexos' 'Mujeres' 'Varones']
['total_grupo' '0 - 9' '10 - 14' '15 - 19' '20 - 24' '25 - 34' '35 - 44'
 '45 - 54' '55 - 64' '65 o m\xa0s']


In [11]:


total_data.columns = total_data.columns.str.strip()
total_data.head()
mapeo_binario = {
    'mujeres': 1,
    'varones': 0,
    'ambos sexos': 2  
}
total_data['sexo_codificado'] = total_data['sexo'].map(mapeo_binario)
print(total_data[['sexo', 'sexo_codificado']].head())



          sexo  sexo_codificado
0  ambos sexos                2
1  ambos sexos                2
2  ambos sexos                2
3  ambos sexos                2
4  ambos sexos                2


In [9]:
total_data['Gender'] = total_data['Gender'].map({
    'male': True,
    'female': False
})

total_data['Admission Status'] = total_data['Admission Status'].map({
    'admitted': True,
    'rejected': False
})

KeyError: 'Gender'

In [ ]:
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
# Ver todos los registros de la columna 'Nombre'
print("Todos los registros de la columna 'Nombre':")
print(total_data['Admission Status'])


In [ ]:
print(total_data['Gender'].unique())
print(total_data['Admission Status'].unique())

In [ ]:
total_data['Admission Status'] = total_data['Admission Status'].fillna(True)

In [ ]:
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
# Ver todos los registros de la columna 'Nombre'
print("Todos los registros de la columna 'Nombre':")
print(total_data['Gender'])

In [ ]:
total_data.head()

In [ ]:
from sklearn.model_selection import train_test_split

num_variables = ["Age", "Gender", "Admission Test Score", "High School Percentage"]

# Dividimos el conjunto de datos en muestras de train y test
X = total_data.drop("Admission Status", axis = 1)[num_variables]
y = total_data["Admission Status"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train.head()

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

In [ ]:
## Reporte de clasificación
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Hiperparámetros
rf_random = {
    "n_estimators": np.arange(10, 100, 10),
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": np.arange(2, 20, 2),
    "min_samples_leaf": np.arange(1, 20, 2),
    "max_features": ["sqrt", "log2"]
}

# Definir el clasificador base
rf = RandomForestClassifier(random_state=42)

# Optimización con búsqueda aleatoria
rs_model = RandomizedSearchCV(
    estimator=rf,
    param_distributions=rf_random,
    n_iter=20,
    cv=5,
    scoring="accuracy",  # No existe "sqrt" como scoring
    verbose=1,
    random_state=42
)

# Entrenamiento
rs_model.fit(X_train, y_train)

In [ ]:
rs_model.best_params_

In [ ]:
### Entrenamiento del mejor modelo


rs_model_best = RandomForestClassifier(**rs_model.best_params_)
rs_model_best.fit(X_train, y_train)

In [ ]:
y_pred_rs = rs_model_best.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred_rs)
accuracy

In [ ]:
from pickle import dump

dump(model, open("../models/decision_tree_classifier_default_42.sav", "wb"))